# 动态规划

动态规划对状态空间的遍历构成一张有向无环图，遍历顺序就是该有向无环图的一个拓扑排序。<br>
有向无环图中的节点对应问题中的“状态”，图中的边则对应状态之间的“转移”，转移的选取就是动态规划中的“决策”。阶段对应对该图进行广搜的每一层。

1、重叠子问题：动态规划算法把原问题视作若干重叠子问题的逐层递进，每个子问题的求解过程都构成一个“阶段”。在完成一个阶段计算后，才会执行下一阶段的计算。<br>
2、无后效性：为了保证计算能够按顺序、不重复地进行，动态规划要求已经求解的子问题不受后续阶段的影响。这个条件就是“无后效性”。<br>
3、最优子结构：很多时候，动态规划用于求解最优化问题。此时，下一阶段的最优解应该能由前面各阶段子问题的最优解导出。也就是说，动态规划在阶段计算完成时，只会在每个状态上保留于最终解集相关的部分代表信息，这些代表信息应该具有可重复的求解过程，并能够导出后续阶段的代表信息

## 线性DP

线性DP（作用在线性空间上的递推）:DP的阶段沿着各个维度线性增长，从一个或多个“边界点”开始有方向地向整个状态空间转移、扩展，最后每个状态上都保留了以自身为“目标”的子问题最优解。

### 最长公共子序列 https://leetcode-cn.com/problems/longest-common-subsequence/

In [ ]:
#带打印方案（不用保存每一步的状态，只需保存代号再通过解析代号还原方案，不单节省空间，更节省时间）
class Solution:
    def longestCommonSubsequence(self, text1: str, text2: str) -> int:
        n=len(text1)
        m=len(text2)
        dp=[[0]*(m+1) for _ in range(n+1)]
        fr=[[0]*(m+1) for _ in range(n+1)]
        for i in range(n):
            for j in range(m):
                if text1[i]==text2[j]:
                    dp[i+1][j+1]=dp[i][j]+1
                else:
                    if dp[i][j+1]>dp[i+1][j]:
                        dp[i+1][j+1]=dp[i][j+1]
                        fr[i+1][j+1]=1
                    else:
                        dp[i+1][j+1]=dp[i+1][j]
                        fr[i+1][j+1]=2
        self.res=[]
        self.dfs(text1,fr,n,m)
        print("".join(self.res))
        return dp[n][m]
    def dfs(self,text,fr,i,j):
        if i==0 or j==0:
            return
        if fr[i][j]==0:
            self.dfs(text,fr,i-1,j-1)
            self.res.append(text[i-1])
        elif fr[i][j]==1:
            self.dfs(text,fr,i-1,j)
        else:
            self.dfs(text,fr,i,j-1)


## 环形DP

环形DP vs 线性DP:<br>
“线性问题”仅比“环形问题”少了一种情况，即尾部和头部相连的情况

打家劫舍 II 

In [ ]:
#2次DP
class Solution:
    def rob(self, nums: List[int]) -> int:
        n=len(nums)
        if n==1: #边界条件
            return nums[0]
        #第一间选
        dp=[[0]*2 for _ in range(n)]
        dp[0][1]=nums[0]
        for i in range(1,n):
            dp[i][0]=max(dp[i-1])
            dp[i][1]=dp[i-1][0]+nums[i]
        ans=dp[n-1][0]
        #第一间不选
        dp=[[0]*2 for _ in range(n)]
        for i in range(1,n):
            dp[i][0]=max(dp[i-1])
            dp[i][1]=dp[i-1][0]+nums[i]
        ans=max(dp[-1]+[ans])
        return ans

## 区间DP

线性DP一般从初态开始，沿着阶段的扩张向某个方向推进，直至计算出目标状态。<br>
区间DP也属于线性DP中的一种，它以“区间长度”作为DP的“阶段”，使用两个坐标（区间的左、右端点）描述每个维度。<br>
在区间DP中，一个状态由若干个比它更小且包含于它的区间所代表的状态转移而来，故区间DP的决策往往就是划分区间的方法。区间DP的初态一般由长度为1的“元区间”构成。<br>
这种向下划分、向上递推的模式与某些树形结构（线段树）类似。

### 1068. 环形石子合并 https://www.acwing.com/problem/content/1070/

In [ ]:
#环形复制策略+区间DP
n=int(input())
a=input()
lst=[int(i) for i in a.split()]
lst=lst*2
ansmin=float("inf")
ansmax=-float("inf")
l=2*n
arr=[0]*l
dpmin=[[float("inf")]*(l) for _ in range(l)]
dpmax=[[0]*(l) for _ in range(l)]
for i in range(l):
    arr[i]=arr[i-1]+lst[i]
for ln in range(1,n+1):
    for left in range(1,l-ln+1):
        right=left+ln-1
        if ln==1:
            dpmin[left][right]=0
            continue
        for k in range(left,right):
            dpmin[left][right]=min(dpmin[left][right],dpmin[left][k]+dpmin[k+1][right]+arr[right]-arr[left-1])
            dpmax[left][right]=max(dpmax[left][right],dpmax[left][k]+dpmax[k+1][right]+arr[right]-arr[left-1])
for i in range(1,n+1):
    ansmin=min(ansmin,dpmin[i][i+n-1])
    ansmax=max(ansmax,dpmax[i][i+n-1])
print(ansmin)
print(ansmax)

## 树形DP

在树上设计动态规划算法时，一般以节点从深到浅（子树丛小到大）的顺序为DP的“阶段”。DP的状态表示中，第一维通常是节点编号（代表以该节点为根的子树）。<br>
大多数时候，采用递归的方式实现树形动态规划，对于每个节点x，先递归它的每个子节点并进行DP,回溯时，从子节点向节点x进行状态转移。<br>
与深搜和广搜一样，除了自顶向下递归外，也可以使用自底部向上拓扑排序来执行树形DP,另外也可使用栈来实现非递归写法（后序遍历）。这三个方法，递归相对较好写。

### 打家劫舍 III

In [ ]:
#递归方式
class Solution:
    def rob(self, root: TreeNode) -> int:
        self.dp={}
        return self.__rob(root)
    def __rob(self,root):
        if root:
            self.dp[root]=[0,root.val]
            if root.left:
                self.dp[root][0]+=self.__rob(root.left)
                self.dp[root][1]+=self.dp[root.left][0]
            if root.right:
                self.dp[root][0]+=self.__rob(root.right)
                self.dp[root][1]+=self.dp[root.right][0]
            return max(self.dp[root])
        return 0

In [ ]:
#拓扑排序
class Solution:
    def rob(self, root: TreeNode) -> int:
        dp={}
        deg={}
        father={}
        queue=[root]
        q=[]
        while queue:
            cur=queue.pop()
            if cur.left:
                deg[cur]=deg.get(cur,0)+1
                queue.append(cur.left)
                father[cur.left]=cur
            if cur.right:
                deg[cur]=deg.get(cur,0)+1
                queue.append(cur.right)
                father[cur.right]=cur
            if cur not in deg:
                q.append(cur)
        while q:
            temp=q.pop()
            dp[temp]=[0,temp.val]
            if temp.left:
                dp[temp][0]+=max(dp[temp.left])
                dp[temp][1]+=dp[temp.left][0]
            if temp.right:
                dp[temp][0]+=max(dp[temp.right])
                dp[temp][1]+=dp[temp.right][0]
            if temp in father:
                deg[father[temp]]-=1
                if deg[father[temp]]==0:
                    q.append(father[temp])
        return max(dp[root])

In [ ]:
#使用栈实现非递归写法（后序遍历）
class Solution:
    def rob(self, root: TreeNode) -> int:
        self.dp={}
        stack=[]
        cur=root
        preNode=None
        while cur or stack:
            while cur:
                stack.append(cur)
                cur=cur.left
            temp=stack.pop()
            if not temp.right or temp.right==preNode:
                self.dp[temp]=[0,temp.val]
                if temp.left:
                    self.dp[temp][0]+=max(self.dp[temp.left])
                    self.dp[temp][1]+=self.dp[temp.left][0]
                if temp.right:
                    self.dp[temp][0]+=max(self.dp[temp.right])
                    self.dp[temp][1]+=self.dp[temp.right][0]
                preNode=temp
            else:
                stack.append(temp)
                cur=temp.right
        return max(self.dp[root])

# 字典树

Trie树的核心思想是空间换时间，典型应用是用于统计和排序大量的字符串（但不仅限于字符串），经常被搜索引擎系统用于文本词频统计。

它的优点是:最大限度地减少无谓的字符串比较，查询效率比哈希表高。

分组思想——前缀相同的字符串在同一子树中<br>
利用字符串的公共前缀来降低查询时间的开销以达到提高效率的目的


### 208. 实现 Trie (前缀树) https://leetcode-cn.com/problems/implement-trie-prefix-tree/

In [ ]:
class Node:
    def __init__(self) -> None:
        self.child={}
        self.count=0
class Trie:
    def __init__(self):
        """
        Initialize your data structure here.
        """
        self.root=Node()

    def insert(self, word: str) -> None:
        """
        Inserts a word into the trie.
        """
        self.find(word,True,False)

    def search(self, word: str) -> bool:
        """
        Returns if the word is in the trie.
        """
        return self.find(word,False,False)

    def startsWith(self, prefix: str) -> bool:
        """
        Returns if there is any word in the trie that starts with the given prefix.
        """
        return self.find(prefix,False,True)
    def find(self,word,isInsert,isPefix):
        cur=self.root
        for s in word:
            if (s not in cur.child):
                if isInsert:
                    cur.child[s]=Node()
                else:
                    return False
            cur=cur.child[s]
        if isInsert:
            cur.count+=1
            return
        if isPefix:
            return True
        return cur.count>0

将insert,search,startsWith抽象出find是自己做的时候没考虑到的，以后碰到重复代买，还需要多思考能否进一步重构。

### 212. 单词搜索 II https://leetcode-cn.com/problems/word-search-ii/

In [ ]:
class Node:
    def __init__(self) -> None:
        self.child={}
        self.word=""
class Trie:
    def __init__(self) -> None:
        self.root=Node()
    def insert(self,word):
        cur=self.root
        for s in word:
            if s not in cur.child:
                cur.child[s]=Node()
            cur=cur.child[s]
        cur.word=word
class Solution:
    def findWords(self, board: List[List[str]], words: List[str]) -> List[str]:
        self.res=[]
        tire=Trie()
        for word in words:
            tire.insert(word)
        self.row=len(board)
        self.col=len(board[0])
        self.visit=[[False]*self.col for _ in range(self.row)]
        self.board=board
        for i in range(self.row):
            for j in range(self.col):
                self.visit[i][j]=True
                self.dfs(i,j,tire.root)
                self.visit[i][j]=False
        return self.res
    def dfs(self,i,j,cur):
        s=self.board[i][j]
        if s not in cur.child:
            return
        father=cur
        cur=cur.child[s]
        if cur.word:
            self.res.append(cur.word)
            cur.word=""
        if not cur.child:
            father.child.pop(s)
        for nx,ny in [(i-1,j),(i+1,j),(i,j-1),(i,j+1)]:
            if nx>=0 and nx<self.row and ny>=0 and ny<self.col:
                if self.visit[nx][ny]==False:
                    self.visit[nx][ny]=True
                    self.dfs(nx,ny,cur)
                    self.visit[nx][ny]=False

这题预习的时候也做过，但时间开销较大6172ms,老师做了如下优化（324ms）：<br>
1、结点上存储单词的额外信息为完整的单词，减少还原单词所需开销<br>
2、visit回溯记得还原<br>
3、为了避免重复，访问过的单词利用dfs特性进行删除。减少重复的方法目前老师教了3种:①是事先判断处理，在3数之后中，排序之后发现与前一个元素一样直接continue；②是在递归中，括号生成，分治划分子问题，注意在划分的时候让2个子问题本身是无重叠的，同样在动态规划中，一个状态的决策之间也需满足不重不漏，也必须保证决策之间的互斥性，从本质上看，二者都是树（无环图），该原理一致；③是事后处理现场，也就是本例，也就是“过河拆桥”，用完了也就可以删除了，避免影响后续分析，带来不必要的时间开销。

# 并查集

连接问题（并查集）vs路径问题（bfs\dfs)<br>
当问题不需要返回确定的路径时，可以选用并查集，效率比深搜和广搜快；<br>
类似堆和搜索树，堆只要最大\最小值，所需信息没有搜索树多，时间效率也更优。

### 省份数量 https://leetcode-cn.com/problems/number-of-provinces/

In [ ]:
class UnoinFind:
    def __init__(self,n) -> None:
        self.parent=[0]*n
        self.size=n
        for i in range(n):
            self.parent[i]=i
    def find(self,p):
        if self.parent[p]!=p:
            self.parent[p]=self.find(self.parent[p])
        return self.parent[p]
    def connect(self,p,q):
        fp=self.find(p)
        fq=self.find(q)
        if fp!=fq:
            self.parent[fp]=fq
            self.size-=1
class Solution:
    def findCircleNum(self, isConnected: List[List[int]]) -> int:
        n=len(isConnected)
        unoin=UnoinFind(n)
        for i in range(n):
            for j in range(i):
                if isConnected[i][j]==1:
                    unoin.connect(i,j)
        return unoin.size

### 130. 被围绕的区域 https://leetcode-cn.com/problems/surrounded-regions/

In [ ]:
class UnionFind:
    def __init__(self,n) -> None:
        self.parent=[0]*n
        for i in range(n):
            self.parent[i]=i
    def connect(self,p,q):
        fp=self.find(p)
        fq=self.find(q)
        if fp!=fq:
            self.parent[fp]=fq
    def find(self,p):
        if self.parent[p]!=p:
            self.parent[p]=self.find(self.parent[p])
        return self.parent[p]
class Solution:
    def solve(self, board: List[List[str]]) -> None:
        """
        Do not return anything, modify board in-place instead.
        """
        r=len(board)
        c=len(board[0])
        union=UnionFind(r*c+1)
        s=set()
        for i in range(r):
            for j in range(c):
                if board[i][j]=="O":
                    if i==0 or j==0 or i==r-1 or j==c-1:
                        union.connect(i*c+j,r*c)
                    else:
                        s.add((i,j))
                    for nx,ny in [(i+1,j),(i,j+1)]:
                        if nx>=0 and nx<r and ny>=0 and ny<c and board[nx][ny]=="O":
                            union.connect(i*c+j,nx*c+ny)
        for i,j in s:
            if union.find(i*c+j)!=union.find(r*c):
                board[i][j]="X"